In [14]:
import csv
from openai import OpenAI
import os
import re

# OPENAI_API_KEYを含むとPushできないため実行時は有効にしてください
# os.environ["OPENAI_API_KEY"] = "*****"

# openAIの機能をclientに代入
client = OpenAI()

# 対象者のMBTIタイプ
user_mbti = "INTJ"

In [15]:
# CSVファイルを読み込み
with open('../output.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    people = list(reader)

In [16]:
def find_best_matches(people, user_mbti, top_n=3):
    prompts = []
    persons = ""
    for person in people:
        persons = persons + str(f"【名前】 {person['Slack表示名']}, 【自己紹介】 {person['自己紹介']}, 【得意な業界】 {person['業界']}, 【関心のある領域】 {person['関心のある領域']}, 【協働で大切にしたいこと】 {person['Tech0のPJTで8期の仲間と協働する際に大切にしたいと思うこと']}, 【プロジェクト推進で得意なこと/苦手なこと】 {person['PJTをする上で自分が得意なこと・苦手なこと']}, 【参加動機や達成したいこと】 {person['Tech0の参加動機と１年後に到達したい・達成したいこと']},") + "\n"
    prompt = f"MBTIタイプが{user_mbti}の人物と合う人物を以下から3名選んでください。対象者は、名前：,選定理由:形式で出力してください\n " + persons


    response =  client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt },
        ],
    )
        
    # コンテンツを抽出
    content = response.choices[0].message.content.strip()
    print(content)

    # 対象者と選定理由を抽出するための正規表現パターン
    pattern = r"対象者は、名前：([^,]+), 選定理由: ([^\n]+)"
    # 正規表現による抽出
    matches = re.findall(pattern, content)
    # 結果を配列に格納
    result = [{"名前": match[0], "選定理由": match[1]} for match in matches]

    return result

In [17]:
# 最適な3人を選択
best_matches = find_best_matches(people, user_mbti)

# 結果を表示
for match in best_matches:
     print(match)

以下の3名をINTJ（内向的直観的思考的判断的）タイプの人物と合うと考え、選定しました。

1. **名前**: Yasuda Rino-8  
   **選定理由**: Yasudaさんは明確な目標を持ち、プロジェクトを当初の目的から逸脱しないよう進行させる得意さがあるため、INTJの論理的かつ計画的に物事を進める性質と合致します。また、ダイナミックな意思決定に苦手意識を持っている点も、より確実な結果を求めるINTJの思考との親和性が見られます。

2. **名前**: Iwamura Hayato-8  
   **選定理由**: Iwamuraさんは新たな価値創出や生産性向上を意識して参加している点から、INTJの未来志向と合わせて考えを深めている姿勢が見受けられます。また、プログラミングを学びたいという意欲は、自己の成長を重視するINTJの特性にも重なります。

3. **名前**: Naito Takuro-8  
   **選定理由**: Naitoさんは新規事業創出において技術スキルを活かしたいと考えており、具体的な目標に向かって計画を立てる姿勢がINTJの思考プロセスと一致します。また、オープンで相談できる関係を築こうとする姿勢も、INTJが目指す合理的で効果的なチーム運営に寄与すると思われます。
